In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st

In [2]:
def price_to_fibo(data,bais,high):
    
    price=data-bais
    fibo=price/abs(bais-high);
    if bais>high:
        fibo=fibo*-1
    return fibo

In [3]:
Path=r'G:\data\data_indicator\xauusd\zigzag\XAUUSD_PERIOD_M1(ZigZag,12,5,3).CSV'
data=pd.read_csv(Path,encoding='utf-16',sep='\t')


In [4]:
n_seq=62
row_data=len(data) - (n_seq - 1)
data_seq=[]
for i in range(row_data):
        data_seq.append(data.loc[ i:i+n_seq-1 ,'zigzag'].tolist())
data_seq=np.array(data_seq)

In [5]:
len_rows=len(data_seq)
len_columns=len(data_seq[0])
data_seq_fibo=[]

for i in range(len_rows):
    for ii in range(len_columns-1):
        x=np.where((data_seq[i]))
        x=np.delete(x,[ii,ii+1])
        data_seq_fibo.append(
            price_to_fibo(
                data_seq[i,x],
                data_seq[i,ii],
                data_seq[i,ii+1]
            )
        )

data_seq_fibo=np.array(data_seq_fibo)  

In [ ]:
mean=np.mean(data_seq_fibo[:,:-2],axis=1)
var=np.var(data_seq_fibo[:,:-2],axis=1)
std=np.std(data_seq_fibo[:,:-2],axis=1)
mode=st.mode(data_seq_fibo[:,:-2] , axis=1)[0]
median=np.median(data_seq_fibo[:,:-2] , axis=1)
max=np.max(data_seq_fibo[:,:-2] , axis=1)
min=np.min(data_seq_fibo[:,:-2] , axis=1)
m_n=max-min
kurtosis=st.kurtosis(data_seq_fibo[:,:-2] , axis=1)
skew=st.skew(data_seq_fibo[:,:-2] , axis=1)
q1=np.percentile(data_seq_fibo[:,:-2],25,axis=1)
q3=np.percentile(data_seq_fibo[:,:-2],75,axis=1)
iqr=q3-q1

amar=np.column_stack([ 
                mean,
                var,
                std,
                mode,
                median,
                max,
                min,
                m_n,
                kurtosis,
                skew,
                q1,
                q3,
                iqr])
    


In [ ]:
print(len(data_seq_fibo))
print(len(amar[0]) + len(data_seq_fibo[0]) )
len(amar[0])

In [ ]:
DF1 = pd.DataFrame(data_seq_fibo)
DF1.to_feather('data_seq_fibo_without_target.feather')
DF2 = pd.DataFrame(amar)
DF2.to_feather('amar.feather')


In [6]:
step=-0.5;
max_target=10 ;
min_target=-10+ step;

x=np.arange(max_target,min_target,step).tolist()
target=np.zeros( [len(x)-1 , 3] )
for i in range(len(x)-1):
    target[i,0]=x[i]
    target[i,1]=x[i+1]
    target[i,2]=i+1

In [7]:
target_col=[58,59]
data_seq_fibo2=data_seq_fibo.copy()

for i in range(len(target_col)):
    data_seq_fibo2=np.hstack((data_seq_fibo2, np.zeros((data_seq_fibo2.shape[0], 1), dtype=data_seq_fibo2.dtype)))
    for ii in range(len(target)):
        condition1= data_seq_fibo2[:,target_col[i]] <= target[ii,0]
        condition2= data_seq_fibo2[:,target_col[i]] >= target[ii,1] 
        x=np.where(condition1 & condition2)
        if(len(x[0]) > 0):
            data_seq_fibo2[x[0],-1]=target[ii,2]

data_seq_fibo2=np.delete(data_seq_fibo2,target_col,1)

In [8]:
DF2 = pd.DataFrame(data_seq_fibo2)

DF2.to_feather('data_seq_fibo_with_target.feather')

In [ ]:
header = True
len
for chunk in DF1:

    chunk.to_csv("ata_seq_fibo_without_target.csv",
        header=header, mode='a')

    header = False